rethnicity: single-character level embedding
---



In [1]:
# change keras default GPU
# os.environ["CUDA_VISIBLE_DEVICES"]="0" # first gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="1" # second gpu
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # runs in cpu

In [2]:
# pip install keras-tuner

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive/rethnicity/

/content/drive/MyDrive/rethnicity


In [5]:
import tensorflow.keras as keras
import tensorflow as tf
print(keras.__version__)
#print(tf.__version__)
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import string

2.6.0


In [6]:
df = pd.read_csv('name_undersampled.csv')

In [7]:
df

name_combine name_first  name_last  race gender
0             Sarah Rahim      Sarah      Rahim     2      F
1                 Amy Mai        Amy        Mai     2      F
2             Sarah Tsang      Sarah      Tsang     2      F
3           Neelima Gopal    Neelima      Gopal     2      F
4              Maria Luna      Maria       Luna     2      F
...                   ...        ...        ...   ...    ...
837051    Michael Ambrose    Michael    Ambrose     5      M
837052     Duane Campbell      Duane   Campbell     5      M
837053  Stephen Rosenfeld    Stephen  Rosenfeld     5      M
837054      Gregory Lynch    Gregory      Lynch     5      M
837055    Geoffrey Coates   Geoffrey     Coates     5      M

[837056 rows x 5 columns]

In [8]:
# create ASCII dictionary
chars = ['E'] + [chr(i) for i in range(97,123)] + [' ', 'U']
id2char = {i:j for i,j in enumerate(chars)}
char2id = {j:i for i,j in enumerate(chars)}

In [9]:
# the characters here are all ASCII, good
# for name in df['name_combine'].tolist():
#     namechars = list(name)
#     for nc in namechars:
#         if nc not in char2id:
#             print(nc)

In [10]:
def name2id(name, l = 10):
    ids = [0] * l # default to 'E'
    for i, c in enumerate(name):
        if i < l:
            if c.isalpha():
                ids[i] = char2id.get(c, char2id['U'])
            elif c in string.punctuation:
                ids[i] = char2id.get(c, char2id[' '])
            else:
                ids[i] = char2id.get(c, char2id['U'])
    return ids

In [11]:
X = [name2id(ln.lower()) for ln in df['name_last']]
y = [int(i) for i in df['race'].tolist()]

In [12]:
# X = df['name_last'].tolist()
# y = [int(i) for i in df['race'].tolist()]

In [13]:
# convert the input (X) from tokens to ids
# X = [[char2id.get(i, char2id['UNK']) for i in each.lower()] for each in X]

In [14]:
# convert the output (y) from 2-5 to 0-3
y = [i-2 for i in y]

In [15]:
# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
# check the distribution
print('distribution of all data points', {i: y.count(i) for i in set(y)})
print('distribution of all training data', {i: y_train.count(i) for i in set(y)})
print('distribution of all testing data', {i: y_test.count(i) for i in set(y)})

distribution of all data points {0: 209264, 1: 209264, 2: 209264, 3: 209264}
distribution of all training data {0: 167403, 1: 167360, 2: 167324, 3: 167557}
distribution of all testing data {0: 41861, 1: 41904, 2: 41940, 3: 41707}


# Train the model

In [17]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM, Bidirectional

num_words = len(id2char)
feature_len = 10 # cut texts after this number of words (among top max_features most common words)
batch_size = 512

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = 4 # np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

669644 train sequences
167412 test sequences
Pad sequences (samples x time)
X_train shape: (669644, 10)
X_test shape: (167412, 10)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (669644, 4)
y_test shape: (167412, 4)


In [18]:
# simple train-test
# first build
model = Sequential()
model.add(Embedding(num_words, 256, input_length=feature_len))
# try out bi-directional LSTM
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.2)))
model.add(Bidirectional(LSTM(512, dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))

# choose between learning rates
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# train model
model.fit(X_train, y_train, batch_size=batch_size, epochs=40, validation_split=0.2, verbose=1, callbacks=[callback])
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

Epoch 1/40
1047/1047 [==============================] - 254s 231ms/step - loss: 0.9806 - accuracy: 0.5793 - val_loss: 0.8796 - val_accuracy: 0.6314
Epoch 2/40
1047/1047 [==============================] - 244s 233ms/step - loss: 0.8381 - accuracy: 0.6602 - val_loss: 0.7960 - val_accuracy: 0.6852
Epoch 3/40
1047/1047 [==============================] - 244s 233ms/step - loss: 0.7725 - accuracy: 0.6990 - val_loss: 0.7561 - val_accuracy: 0.7074
Epoch 4/40
1047/1047 [==============================] - 244s 233ms/step - loss: 0.7304 - accuracy: 0.7214 - val_loss: 0.7367 - val_accuracy: 0.7198
Epoch 5/40
1047/1047 [==============================] - 244s 233ms/step - loss: 0.6987 - accuracy: 0.7363 - val_loss: 0.7246 - val_accuracy: 0.7275
Epoch 6/40
1047/1047 [==============================] - 244s 233ms/step - loss: 0.6741 - accuracy: 0.7470 - val_loss: 0.7189 - val_accuracy: 0.7300
Epoch 7/40
1047/1047 [==============================] - 244s 233ms/step - loss: 0.6524 - accuracy: 0.7557 - val_

In [19]:
# now lets' test
y_pred = model.predict(X_test, batch_size=batch_size, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(np.argmax(y_test, axis=1), y_pred_bool))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred_bool))

327/327 [==============================] - 26s 75ms/step
              precision    recall  f1-score   support

           0       0.85      0.70      0.77     41861
           1       0.64      0.83      0.72     41904
           2       0.84      0.85      0.84     41940
           3       0.63      0.55      0.59     41707

    accuracy                           0.73    167412
   macro avg       0.74      0.73      0.73    167412
weighted avg       0.74      0.73      0.73    167412

[[29444  4078  3205  5134]
 [ 1187 34596   917  5204]
 [ 1560  1753 35662  2965]
 [ 2607 13603  2730 22767]]


In [20]:
model.save('lastname.h5', include_optimizer=False)
model.save('lastname_opt.h5')

# Distill the Model

In [21]:
# TODO: shall we try model distillation for compressing the model size?
# so that we will have smaller model to work with
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [22]:
# Create the student
student = keras.Sequential(
    [
        Embedding(num_words, 32, input_length=feature_len),
        Bidirectional(LSTM(16, return_sequences=True, dropout=0.2), 
                            input_shape=(feature_len, 16)),
        Bidirectional(LSTM(16, dropout=0.2)),
        Dense(num_classes, activation='softmax')
    ],
    name="student",
)

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(X_train, y_train, epochs=30)

# Evaluate student on test dataset
distiller.evaluate(X_test, y_test)

Epoch 1/30
20927/20927 [==============================] - 414s 19ms/step - categorical_accuracy: 0.5751 - student_loss: 0.9866 - distillation_loss: 3.1864e-04
Epoch 2/30
20927/20927 [==============================] - 412s 20ms/step - categorical_accuracy: 0.6204 - student_loss: 0.9078 - distillation_loss: 2.6282e-04
Epoch 3/30
20927/20927 [==============================] - 412s 20ms/step - categorical_accuracy: 0.6328 - student_loss: 0.8861 - distillation_loss: 2.4726e-04
Epoch 4/30
20927/20927 [==============================] - 412s 20ms/step - categorical_accuracy: 0.6387 - student_loss: 0.8751 - distillation_loss: 2.3956e-04
Epoch 5/30
20927/20927 [==============================] - 414s 20ms/step - categorical_accuracy: 0.6427 - student_loss: 0.8681 - distillation_loss: 2.3428e-04
Epoch 6/30
20927/20927 [==============================] - 412s 20ms/step - categorical_accuracy: 0.6457 - student_loss: 0.8632 - distillation_loss: 2.3066e-04
Epoch 7/30
20927/20927 [======================

[0.6715946197509766, 0.8687471151351929]

In [ ]:
y_pred = distiller.student.predict(X_test, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(np.argmax(y_test, axis=1), y_pred_bool))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred_bool))

In [ ]:
distiller.student.save('lastname_distill_opt.h5')
distiller.student.save('lastname_distill.h5', include_optimizer=False)

# Tune the Model

In [ ]:
model.summary()

In [ ]:
model.get_config()

In [ ]:
model.optimizer

In [ ]:
from keras import backend as K
K.eval(model.optimizer.lr)

In [ ]:
# from google.colab import files
# files.download('fullname_2_ethnicity_bilstm.h5') 